In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import LogNorm
import pandas as pd
import statsmodels.api as sm

In [2]:
path0 = r'E:\netData\SciSciNet\2000-2015\paper2000-2015.tsv.zip'
path1 = r'E:\netData\SciSciNet\2000-2015\matchedNIH2000-2015.tsv.zip'
path2 = r'E:\netData\SciSciNet\2000-2015\matchedNSF2000-2015.tsv.zip'

In [3]:
df1 = pd.read_csv(path1)
df2 = pd.read_csv(path2)

C:\Users\nd\AppData\Local\Temp\ipykernel_13388\297883942.py:1: DtypeWarning: Columns (26,29) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv(path1)


In [4]:
df1

,PaperID,Year,FieldID,JournalID,Reference_Count,C_f,C5,Team_Size,Disruption,Atyp_10pct_Z,...,Institution_Count,NIH_Count,NSF_Count,Volume,Issue,FirstPage,LastPage,NIH,NSF,PageLength
0,1816482797,2015,86803240,166515463,29,1.305137,15,7,0.000106,-4.246876,...,3,1,0,24,21,5995,6002,1,0,7
1,1824457652,2015,86803240,166515463,65,3.371604,50,21,-0.004329,-7.690156,...,5,0,0,24,21,5965,5976,0,0,11
2,1893201598,2015,86803240,166515463,28,3.534746,52,11,-0.007772,19.962751,...,3,0,0,24,21,5977,5984,0,0,7
3,1875600228,2015,86803240,166515463,40,0.815711,14,8,-0.000523,-6.721229,...,2,2,0,24,21,6093,6105,1,0,12
4,1503301128,2015,86803240,166515463,67,2.283990,35,10,-0.000589,-0.049458,...,3,0,0,24,21,6066,6079,0,0,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5546780,2165646896,2004,86803240,145090216,17,0.244307,9,3,0.000314,3.186190,...,1,0,0,36,4,250,258,0,0,8
5546781,2156997304,2004,86803240,145090216,16,0.510824,8,4,-0.001564,85.732503,...,1,0,0,36,4,284,289,0,0,5
5546782,2104663756,2004,86803240,145090216,18,0.333146,8,3,-0.002029,-3.352016,...,1,0,0,36,4,269,276,0,0,7
5546783,2131685436,2004,86803240,145090216,47,0.111049,4,4,-0.001212,10.599674,...,1,0,0,36,4,243,249,0,0,6


# rescale

In [5]:
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# df1['Atyp_10pct_Z'] = scaler.fit_transform(df1['Atyp_10pct_Z'].values.reshape(-1, 1)).flatten()

In [6]:
df1['nAtyp_10pct_Z'] = df1['Atyp_10pct_Z']*(-1)
df1['nAtyp_Median_Z']= df1['Atyp_Median_Z']*(-1)

In [7]:
df2['nAtyp_10pct_Z'] = df2['Atyp_10pct_Z']*(-1)
df2['nAtyp_Median_Z']= df2['Atyp_Median_Z']*(-1)

# statistics

In [8]:
columns = ['C5', 'C_f', 'Patent_Count', 'nAtyp_10pct_Z','nAtyp_Median_Z', 'Disruption','Reference_Count', 'Atyp_Pairs', 'Team_Size','Institution_Count']

In [9]:
dff = df1[columns].describe().T
dff['skewness'] = df1[columns].skew()
dff['kurt'] = df1[columns].kurt()
dff.to_excel(r'G:\A J YANG PAPER\All paper analysis\JIF Unveiling Funded Research Novelty and Disruptive Potential\Tables\Table 1\1.xlsx')

In [10]:
dff = df2[columns].describe().T
dff['skewness'] = df2[columns].skew()
dff['kurt'] = df2[columns].kurt()
dff.to_excel(r'G:\A J YANG PAPER\All paper analysis\JIF Unveiling Funded Research Novelty and Disruptive Potential\Tables\Table 1\2.xlsx')

# regression

In [11]:
def save_output(model , savepath):
    params = model.params
    bse = model.bse
    pvalues = model.pvalues
    # create a DataFrame with the results
    results_df = pd.DataFrame({'coef': params,'std err': bse,'p_value': pvalues})
    results_df = results_df.round(4)
    # add asterisks to the p-values based on the significance level
    results_df['p_value'] = results_df['p_value'].apply(lambda x: '{:.3f}'.format(x))
    results_df['p_value'] = results_df['p_value'].astype(float)
    results_df['p_star'] = ''
    results_df.loc[results_df['p_value'] < 0.01, 'p_star'] = '***'
    results_df.loc[(results_df['p_value'] >= 0.01) & (results_df['p_value'] < 0.05), 'p_star'] = '**'
    results_df.loc[(results_df['p_value'] >= 0.05) & (results_df['p_value'] < 0.1), 'p_star'] = '*'
    results_df['p_value'] = results_df['p_value'].apply(lambda x: '{:.3f}'.format(x))
    results_df['v'] = results_df['coef'].astype(str) + results_df['p_star'] +'\n'+ '('+ results_df['std err'].astype(str) +')'
    results_df.to_excel(savepath)

# Impact NIH

In [39]:
path_i = 'G:/A J YANG PAPER/All paper analysis/JIF Unveiling Funded Research Novelty and Disruptive Potential/Tables/Table 2'

In [13]:
formula = 'C5 ~  NIH + Reference_Count + Atyp_Pairs + Team_Size + Institution_Count + C(Year) + C(FieldID)'

model = sm.GLM.from_formula(formula, data=df1, family=sm.families.Poisson()).fit()
print(model.summary())
save_output(model , path_i + '/10.xlsx')

                 Generalized Linear Model Regression Results                  
Dep. Variable:                     C5   No. Observations:              5546785
Model:                            GLM   Df Residuals:                  5546745
Model Family:                 Poisson   Df Model:                           39
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -9.9929e+07
Date:                Wed, 14 Jun 2023   Deviance:                   1.7543e+08
Time:                        19:15:01   Pearson chi2:                 5.83e+08
No. Iterations:                    35   Pseudo R-squ. (CS):             0.9834
Covariance Type:            nonrobust                                         
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                 

In [14]:
formula = 'C_f ~  NIH + Reference_Count + Atyp_Pairs + Team_Size + Institution_Count + C(Year) + C(FieldID)'

model = sm.formula.ols(formula=formula, data=df1).fit()
print(model.summary())
save_output(model , path_i + '/11.xlsx')

                            OLS Regression Results                            
Dep. Variable:                    C_f   R-squared:                       0.045
Model:                            OLS   Adj. R-squared:                  0.045
Method:                 Least Squares   F-statistic:                     6749.
Date:                Wed, 14 Jun 2023   Prob (F-statistic):               0.00
Time:                        19:15:36   Log-Likelihood:            -1.9250e+07
No. Observations:             5546785   AIC:                         3.850e+07
Df Residuals:                 5546745   BIC:                         3.850e+07
Df Model:                          39                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                 

In [15]:
formula = 'Patent_Count ~  NIH + Reference_Count + Atyp_Pairs + Team_Size + Institution_Count + C(Year) + C(FieldID)'

model = sm.formula.ols(formula=formula, data=df1).fit()
print(model.summary())
save_output(model , path_i + '/12.xlsx')

                            OLS Regression Results                            
Dep. Variable:           Patent_Count   R-squared:                       0.011
Model:                            OLS   Adj. R-squared:                  0.011
Method:                 Least Squares   F-statistic:                     1596.
Date:                Wed, 14 Jun 2023   Prob (F-statistic):               0.00
Time:                        19:16:10   Log-Likelihood:            -2.1506e+07
No. Observations:             5546785   AIC:                         4.301e+07
Df Residuals:                 5546745   BIC:                         4.301e+07
Df Model:                          39                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                 

In [16]:
formula = 'C5 ~  NIH + C(Year) + C(FieldID)'

model = sm.GLM.from_formula(formula, data=df1, family=sm.families.Poisson()).fit()
print(model.summary())
save_output(model , path_i + '/20.xlsx')

                 Generalized Linear Model Regression Results                  
Dep. Variable:                     C5   No. Observations:              5546785
Model:                            GLM   Df Residuals:                  5546749
Model Family:                 Poisson   Df Model:                           35
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -1.0753e+08
Date:                Wed, 14 Jun 2023   Deviance:                   1.9063e+08
Time:                        19:24:30   Pearson chi2:                 6.54e+08
No. Iterations:                    67   Pseudo R-squ. (CS):             0.7429
Covariance Type:            nonrobust                                         
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                 

In [17]:
formula = 'C_f ~  NIH + C(Year) + C(FieldID)'

model = sm.formula.ols(formula=formula, data=df1).fit()
print(model.summary())
save_output(model , path_i + '/21.xlsx')

                            OLS Regression Results                            
Dep. Variable:                    C_f   R-squared:                       0.011
Model:                            OLS   Adj. R-squared:                  0.011
Method:                 Least Squares   F-statistic:                     1778.
Date:                Wed, 14 Jun 2023   Prob (F-statistic):               0.00
Time:                        19:25:05   Log-Likelihood:            -1.9347e+07
No. Observations:             5546785   AIC:                         3.869e+07
Df Residuals:                 5546749   BIC:                         3.870e+07
Df Model:                          35                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                 

In [18]:
formula = 'Patent_Count ~  NIH + C(Year) + C(FieldID)'

# model = sm.GLM.from_formula(formula, data=df1, family=sm.families.Poisson()).fit()
model = sm.formula.ols(formula=formula, data=df1).fit()
print(model.summary())
save_output(model , path_i + '/22.xlsx')

                            OLS Regression Results                            
Dep. Variable:           Patent_Count   R-squared:                       0.010
Model:                            OLS   Adj. R-squared:                  0.010
Method:                 Least Squares   F-statistic:                     1547.
Date:                Wed, 14 Jun 2023   Prob (F-statistic):               0.00
Time:                        19:25:37   Log-Likelihood:            -2.1510e+07
No. Observations:             5546785   AIC:                         4.302e+07
Df Residuals:                 5546749   BIC:                         4.302e+07
Df Model:                          35                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                 

# Impact NSF

In [19]:
formula = 'C5 ~  NIH + Reference_Count + Atyp_Pairs + Team_Size + Institution_Count + C(Year) + C(FieldID)'

model = sm.GLM.from_formula(formula, data=df2, family=sm.families.Poisson()).fit()
print(model.summary())
save_output(model , path_i + '/s10.xlsx')

                 Generalized Linear Model Regression Results                  
Dep. Variable:                     C5   No. Observations:              3515356
Model:                            GLM   Df Residuals:                  3515316
Model Family:                 Poisson   Df Model:                           39
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -6.0214e+07
Date:                Wed, 14 Jun 2023   Deviance:                   1.0538e+08
Time:                        19:26:56   Pearson chi2:                 3.83e+08
No. Iterations:                    11   Pseudo R-squ. (CS):             0.9871
Covariance Type:            nonrobust                                         
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                 

In [20]:
formula = 'C_f ~  NIH + Reference_Count + Atyp_Pairs + Team_Size + Institution_Count + C(Year) + C(FieldID)'

model = sm.formula.ols(formula=formula, data=df2).fit()
print(model.summary())
save_output(model , path_i + '/s11.xlsx')

                            OLS Regression Results                            
Dep. Variable:                    C_f   R-squared:                       0.040
Model:                            OLS   Adj. R-squared:                  0.040
Method:                 Least Squares   F-statistic:                     3757.
Date:                Wed, 14 Jun 2023   Prob (F-statistic):               0.00
Time:                        19:27:18   Log-Likelihood:            -1.2586e+07
No. Observations:             3515356   AIC:                         2.517e+07
Df Residuals:                 3515316   BIC:                         2.517e+07
Df Model:                          39                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                 

In [21]:
formula = 'Patent_Count ~  NIH + Reference_Count + Atyp_Pairs + Team_Size + Institution_Count + C(Year) + C(FieldID)'

model = sm.formula.ols(formula=formula, data=df2).fit()
print(model.summary())
save_output(model , path_i + '/s12.xlsx')

                            OLS Regression Results                            
Dep. Variable:           Patent_Count   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     405.2
Date:                Wed, 14 Jun 2023   Prob (F-statistic):               0.00
Time:                        19:27:39   Log-Likelihood:            -1.3478e+07
No. Observations:             3515356   AIC:                         2.696e+07
Df Residuals:                 3515316   BIC:                         2.696e+07
Df Model:                          39                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                 

In [22]:
formula = 'C5 ~  NIH + C(Year) + C(FieldID)'

model = sm.GLM.from_formula(formula, data=df2, family=sm.families.Poisson()).fit()
print(model.summary())
save_output(model , path_i + '/s20.xlsx')

                 Generalized Linear Model Regression Results                  
Dep. Variable:                     C5   No. Observations:              3515356
Model:                            GLM   Df Residuals:                  3515320
Model Family:                 Poisson   Df Model:                           35
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -6.5046e+07
Date:                Wed, 14 Jun 2023   Deviance:                   1.1505e+08
Time:                        19:30:49   Pearson chi2:                 4.30e+08
No. Iterations:                    40   Pseudo R-squ. (CS):             0.7981
Covariance Type:            nonrobust                                         
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                 

In [23]:
formula = 'C_f ~  NIH + C(Year) + C(FieldID)'

model = sm.formula.ols(formula=formula, data=df2).fit()
print(model.summary())
save_output(model , path_i + '/s21.xlsx')

                            OLS Regression Results                            
Dep. Variable:                    C_f   R-squared:                       0.014
Model:                            OLS   Adj. R-squared:                  0.014
Method:                 Least Squares   F-statistic:                     1389.
Date:                Wed, 14 Jun 2023   Prob (F-statistic):               0.00
Time:                        19:31:10   Log-Likelihood:            -1.2633e+07
No. Observations:             3515356   AIC:                         2.527e+07
Df Residuals:                 3515320   BIC:                         2.527e+07
Df Model:                          35                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                 

In [40]:
formula = 'Patent_Count ~  NIH + C(Year) + C(FieldID)'

model = sm.formula.ols(formula=formula, data=df2).fit()
print(model.summary())
save_output(model , path_i + '/s22.xlsx')

                            OLS Regression Results                            
Dep. Variable:           Patent_Count   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     434.7
Date:                Wed, 14 Jun 2023   Prob (F-statistic):               0.00
Time:                        20:22:38   Log-Likelihood:            -1.3479e+07
No. Observations:             3515356   AIC:                         2.696e+07
Df Residuals:                 3515320   BIC:                         2.696e+07
Df Model:                          35                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                 

# Innovation NIH

In [41]:
path_i = 'G:/A J YANG PAPER/All paper analysis/JIF Unveiling Funded Research Novelty and Disruptive Potential/Tables/Table 3'

In [26]:
formula = 'nAtyp_10pct_Z ~  NIH + Reference_Count + Atyp_Pairs + Team_Size + Institution_Count + C(Year) + C(FieldID)'

model = sm.formula.ols(formula=formula, data=df1).fit()
print(model.summary())
save_output(model , path_i + '/10.xlsx')

                            OLS Regression Results                            
Dep. Variable:          nAtyp_10pct_Z   R-squared:                       0.019
Model:                            OLS   Adj. R-squared:                  0.019
Method:                 Least Squares   F-statistic:                     2780.
Date:                Wed, 14 Jun 2023   Prob (F-statistic):               0.00
Time:                        19:32:47   Log-Likelihood:            -3.7625e+07
No. Observations:             5546785   AIC:                         7.525e+07
Df Residuals:                 5546745   BIC:                         7.525e+07
Df Model:                          39                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                 

In [27]:
formula = 'nAtyp_Median_Z ~  NIH + Reference_Count + Atyp_Pairs + Team_Size + Institution_Count + C(Year) + C(FieldID)'

model = sm.formula.ols(formula=formula, data=df1).fit()
print(model.summary())
save_output(model , path_i + '/11.xlsx')

                            OLS Regression Results                            
Dep. Variable:         nAtyp_Median_Z   R-squared:                       0.036
Model:                            OLS   Adj. R-squared:                  0.036
Method:                 Least Squares   F-statistic:                     5378.
Date:                Wed, 14 Jun 2023   Prob (F-statistic):               0.00
Time:                        19:33:22   Log-Likelihood:            -3.9601e+07
No. Observations:             5546785   AIC:                         7.920e+07
Df Residuals:                 5546745   BIC:                         7.920e+07
Df Model:                          39                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                -

In [28]:
formula = 'Disruption ~  NIH + Reference_Count + Atyp_Pairs + Team_Size + Institution_Count + C(Year) + C(FieldID)'

model = sm.formula.ols(formula=formula, data=df1).fit()
print(model.summary())
save_output(model , path_i + '/12.xlsx')

                            OLS Regression Results                            
Dep. Variable:             Disruption   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     550.2
Date:                Wed, 14 Jun 2023   Prob (F-statistic):               0.00
Time:                        19:33:56   Log-Likelihood:             1.5275e+07
No. Observations:             5546785   AIC:                        -3.055e+07
Df Residuals:                 5546745   BIC:                        -3.055e+07
Df Model:                          39                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                 

In [29]:
formula = 'nAtyp_10pct_Z ~  NIH + C(Year) + C(FieldID)'

model = sm.formula.ols(formula=formula, data=df1).fit()
print(model.summary())
save_output(model , path_i + '/20.xlsx')

                            OLS Regression Results                            
Dep. Variable:          nAtyp_10pct_Z   R-squared:                       0.008
Model:                            OLS   Adj. R-squared:                  0.008
Method:                 Least Squares   F-statistic:                     1220.
Date:                Wed, 14 Jun 2023   Prob (F-statistic):               0.00
Time:                        19:34:28   Log-Likelihood:            -3.7657e+07
No. Observations:             5546785   AIC:                         7.531e+07
Df Residuals:                 5546749   BIC:                         7.531e+07
Df Model:                          35                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                 

In [30]:
formula = 'nAtyp_Median_Z ~  NIH + C(Year) + C(FieldID)'

model = sm.formula.ols(formula=formula, data=df1).fit()
print(model.summary())
save_output(model , path_i + '/21.xlsx')

                            OLS Regression Results                            
Dep. Variable:         nAtyp_Median_Z   R-squared:                       0.013
Model:                            OLS   Adj. R-squared:                  0.013
Method:                 Least Squares   F-statistic:                     2122.
Date:                Wed, 14 Jun 2023   Prob (F-statistic):               0.00
Time:                        19:35:00   Log-Likelihood:            -3.9667e+07
No. Observations:             5546785   AIC:                         7.933e+07
Df Residuals:                 5546749   BIC:                         7.933e+07
Df Model:                          35                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                -

In [31]:
formula = 'Disruption ~  NIH + C(Year) + C(FieldID)'

model = sm.formula.ols(formula=formula, data=df1).fit()
print(model.summary())
save_output(model , path_i + '/22.xlsx')

                            OLS Regression Results                            
Dep. Variable:             Disruption   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     348.7
Date:                Wed, 14 Jun 2023   Prob (F-statistic):               0.00
Time:                        19:35:32   Log-Likelihood:             1.5270e+07
No. Observations:             5546785   AIC:                        -3.054e+07
Df Residuals:                 5546749   BIC:                        -3.054e+07
Df Model:                          35                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                 

# Innovation NSF

In [32]:
formula = 'nAtyp_10pct_Z ~  NIH + Reference_Count + Atyp_Pairs + Team_Size + Institution_Count + C(Year) + C(FieldID)'

model = sm.formula.ols(formula=formula, data=df2).fit()
print(model.summary())
save_output(model , path_i + '/s10.xlsx')

                            OLS Regression Results                            
Dep. Variable:          nAtyp_10pct_Z   R-squared:                       0.031
Model:                            OLS   Adj. R-squared:                  0.031
Method:                 Least Squares   F-statistic:                     2931.
Date:                Wed, 14 Jun 2023   Prob (F-statistic):               0.00
Time:                        19:35:53   Log-Likelihood:            -2.5674e+07
No. Observations:             3515356   AIC:                         5.135e+07
Df Residuals:                 3515316   BIC:                         5.135e+07
Df Model:                          39                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                -

In [33]:
formula = 'nAtyp_Median_Z ~  NIH + Reference_Count + Atyp_Pairs + Team_Size + Institution_Count + C(Year) + C(FieldID)'

model = sm.formula.ols(formula=formula, data=df2).fit()
print(model.summary())
save_output(model , path_i + '/s11.xlsx')

                            OLS Regression Results                            
Dep. Variable:         nAtyp_Median_Z   R-squared:                       0.123
Model:                            OLS   Adj. R-squared:                  0.123
Method:                 Least Squares   F-statistic:                 1.263e+04
Date:                Wed, 14 Jun 2023   Prob (F-statistic):               0.00
Time:                        19:36:15   Log-Likelihood:            -2.7281e+07
No. Observations:             3515356   AIC:                         5.456e+07
Df Residuals:                 3515316   BIC:                         5.456e+07
Df Model:                          39                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                -

In [34]:
formula = 'Disruption ~  NIH + Reference_Count + Atyp_Pairs + Team_Size + Institution_Count + C(Year) + C(FieldID)'

model = sm.formula.ols(formula=formula, data=df2).fit()
print(model.summary())
save_output(model , path_i + '/s12.xlsx')

                            OLS Regression Results                            
Dep. Variable:             Disruption   R-squared:                       0.009
Model:                            OLS   Adj. R-squared:                  0.008
Method:                 Least Squares   F-statistic:                     772.7
Date:                Wed, 14 Jun 2023   Prob (F-statistic):               0.00
Time:                        19:36:37   Log-Likelihood:             9.7355e+06
No. Observations:             3515356   AIC:                        -1.947e+07
Df Residuals:                 3515316   BIC:                        -1.947e+07
Df Model:                          39                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                 

In [35]:
formula = 'nAtyp_10pct_Z ~  NIH + C(Year) + C(FieldID)'

model = sm.formula.ols(formula=formula, data=df2).fit()
print(model.summary())
save_output(model , path_i + '/s20.xlsx')

                            OLS Regression Results                            
Dep. Variable:          nAtyp_10pct_Z   R-squared:                       0.023
Model:                            OLS   Adj. R-squared:                  0.023
Method:                 Least Squares   F-statistic:                     2408.
Date:                Wed, 14 Jun 2023   Prob (F-statistic):               0.00
Time:                        19:36:57   Log-Likelihood:            -2.5689e+07
No. Observations:             3515356   AIC:                         5.138e+07
Df Residuals:                 3515320   BIC:                         5.138e+07
Df Model:                          35                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                 

In [36]:
formula = 'nAtyp_Median_Z ~  NIH + C(Year) + C(FieldID)'

model = sm.formula.ols(formula=formula, data=df2).fit()
print(model.summary())
save_output(model , path_i + '/s21.xlsx')

                            OLS Regression Results                            
Dep. Variable:         nAtyp_Median_Z   R-squared:                       0.114
Model:                            OLS   Adj. R-squared:                  0.114
Method:                 Least Squares   F-statistic:                 1.291e+04
Date:                Wed, 14 Jun 2023   Prob (F-statistic):               0.00
Time:                        19:37:17   Log-Likelihood:            -2.7299e+07
No. Observations:             3515356   AIC:                         5.460e+07
Df Residuals:                 3515320   BIC:                         5.460e+07
Df Model:                          35                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                -

In [42]:
formula = 'Disruption ~  NIH + C(Year) + C(FieldID)'

model = sm.formula.ols(formula=formula, data=df2).fit()
print(model.summary())
save_output(model , path_i + '/s22.xlsx')

                            OLS Regression Results                            
Dep. Variable:             Disruption   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                  0.007
Method:                 Least Squares   F-statistic:                     736.2
Date:                Wed, 14 Jun 2023   Prob (F-statistic):               0.00
Time:                        20:22:59   Log-Likelihood:             9.7333e+06
No. Observations:             3515356   AIC:                        -1.947e+07
Df Residuals:                 3515320   BIC:                        -1.947e+07
Df Model:                          35                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                 